## ETL : YELP - User
Este archivo solo carga informacion de la tabla user.
Como pre requisito es necesario:
+ tener la tabla yelp_business_florida, que contiene los establecimientos del estado FLORIDA
+ tener la tabla yelp_review que contiene la informacion de reviews de establec. del estado FLORIDA

Haciendo un cruce de esas dos tablas sabemos que registros de la tabla User debemos considerar en la carga
para el analisis del estado FLORIDA.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def ETL_carga_user(df_user_input, df_yelp_review):
  # YELP : carga de informacion user
  import pandas as pd
  df_user_input = df_user_input[df_user_input['user_id'].isin(df_yelp_review['user_id'])]
  # agregamos la cuenta de amigos y de anios en elite
  df_user_input['friends_count'] = df_user_input['friends'].apply(lambda x: len(x.split(',')))
  df_user_input['elite_count'] = df_user_input['elite'].apply(lambda x: len(x.split(',')))
  df_user_input = df_user_input.drop(['elite','friends'], axis=1)
  #df_yelp_user.to_csv('/content/drive/MyDrive/Otros/yelp_user.csv', index=False)

  # Crear un cliente de BigQuery
    client = bigquery.Client(project="finalprojectprototype-397114")

    # Cargar el DataFrame en BigQuery
    df.to_gbq(destination_table="finalprojectprototype-397114.yelp.dim_yelp_user", project_id="finalprojectprototype-397114", if_exists="append")


In [ ]:
# Cargamos user
import pandas as pd
import pyarrow.parquet as pq
df_yelp_user_orig =pd.DataFrame()
parquet_file = pq.ParquetFile("/content/drive/MyDrive/yelp/Copia de user.parquet")
for i in parquet_file.iter_batches(batch_size=10000):
      df_chunk = i.to_pandas()
      df_yelp_user_orig = pd.concat([df_yelp_user_orig, df_chunk], ignore_index=True)
df_yelp_review = pd.read_csv("/content/drive/MyDrive/Otros/yelp_review_florida.csv")

df_user_salida = ETL_carga_user(df_yelp_user_orig, df_yelp_review)
